In [2]:
BASE_FOLDER = '../../raw_domo_kb'
OUTPUT_FOLDER  ='../../nbs/knowledge_base/'


In [12]:
# !pip install pyyaml

In [209]:
import os
from os import listdir
from os.path import isfile, join


def crawl_directory(folder_path):
  res = []
  for r, d, f in os.walk(folder_path):
    for file in f:
        if file == "process.json":
            res.append(os.path.join(r, file))
  return res


category_ls = crawl_directory(os.path.join(BASE_FOLDER, 'category'))
category_ls[0:5]

['../../raw_domo_kb\\category\\0TO5w000000ZamlGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZammGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZamnGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZamoGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZampGAC\\process.json']

what is the longest breadcrumb?

In [294]:
import re
import json
crumb_ls = []

category_objs = []
for file_path in category_ls:
    with open(file_path, 'r', encoding='utf-8') as f:
        category_objs.append(json.loads(f.read()))

category_objs


def get_crumbs(category_obj):
    for index, crumb in enumerate(category_obj['breadcrumbs']):

        if index == 0:
            path = 'knowledge_base'
            crumb.update({'path': path})
        else:
            path = f"{category_obj['breadcrumbs'][index-1]['path']}/{crumb['text']}"
            crumb.update(
                {'path': path})
        
        crumb.update({'doc_path': f"{path}/index.qmd"})
                 

        crumb_ls.append(crumb)
        del crumb['url']

    crumb_ls[-1].update({'title': category_obj['title']})
    crumb_ls[-1].update({'description' : category_obj['description']})


[get_crumbs(category_obj) for category_obj in category_objs]

# crumb_ls
category_objs[0]

{'file_path': '../../raw_domo_kb/category/0TO5w000000ZamlGAC/index.html',
 'content': {'section-list': [{'text': 'implementing_sso',
    'url': '/s/topic/0TO5w000000ZanoGAC'},
   {'text': 'certifications', 'url': '/s/topic/0TO5w000000ZanGGAS'},
   {'text': 'domo_security_options', 'url': '/s/topic/0TO5w000000ZandGAC'},
   {'text': 'governance_tools', 'url': '/s/topic/0TO5w000000ZannGAC'},
   {'text': 'controlling_access_in_domo',
    'url': '/s/topic/0TO5w000000ZanMGAS'},
   {'text': 'specifying_company_settings',
    'url': '/s/topic/0TO5w000000ZaoGGAS'},
   {'text': 'admin_settings_overview', 'url': '/s/topic/0TO5w000000Zan4GAC'},
   {'text': 'approvals', 'url': '/s/topic/0TO5w000000Zan5GAC'},
   {'text': 'workflows', 'url': '/s/topic/0TO5w000000ZkAMGA0'}]},
 'title': 'Administrating Domo',
 'title_clean': 'administrating_domo',
 'description': 'Learn how to manage users and groups, control access to content, configure security settings, and more.',
 'breadcrumbs': [{'text': 'domo',


In [268]:
from pprint import pprint
from functools import reduce  # forward compatibility for Python 3
import operator


def deep_merge(dict1, dict2):
    result = dict1.copy()
    for key, value in dict2.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = deep_merge(result[key], value)
        else:
            result[key] = value
    return result


def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    return reduce(operator.getitem, items, root)


def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value


def update_f(crumb, dataDict):
    path_list = crumb['path'].split('/')

    set_by_path(dataDict, path_list, {})

    set_by_path(dataDict, path_list+['contents'], crumb['doc_path'])

    if crumb.get('title'):
        set_by_path(dataDict, path_list+['section'], crumb['title'])
    return dataDict


dataDict = {'knowledge_base': {}}
f = {}

for crumb in crumb_ls:
    dataDict = update_f(crumb, dataDict)

    f = deep_merge(f, dataDict)


def flatten_ls(obj):

    res = {'contents': []}

    for key, value in obj.items():
        # print(key, value)
        if isinstance(value, str):

            res.update({key: [value] if key == 'contents' else value})

        if isinstance(value, dict):
            res['contents'].append(flatten_ls(value))

    return res


flat_f = flatten_ls(f)

flat_f = {"section": "knowledge_base", **flat_f['contents'][0]}

pprint(flat_f)


{'contents': ['knowledge_base/index.qmd',
              {'contents': ['knowledge_base/administrating_domo/index.qmd',
                            {'contents': ['knowledge_base/administrating_domo/admin_settings_overview/index.qmd'],
                             'section': 'Admin Settings Overview'},
                            {'contents': ['knowledge_base/administrating_domo/approvals/index.qmd'],
                             'section': 'Approvals'},
                            {'contents': ['knowledge_base/administrating_domo/domo_security_options/index.qmd'],
                             'section': 'Domo Security Options'},
                            {'contents': ['knowledge_base/administrating_domo/certifications/index.qmd'],
                             'section': 'Certifications'},
                            {'contents': ['knowledge_base/administrating_domo/controlling_access_in_domo/index.qmd',
                                          {'contents': ['knowledge_base/administrat

In [291]:
import shutil

def reset_folder(folder_path):
      for r, d, f in os.walk(folder_path):
        for file_name in f:
            file_path = os.path.join(r, file_name) 
            os.remove(file_path)
        
        for directory in d:
            folder_path = os.path.join(r, directory) 
            shutil.rmtree(folder_path)
        

reset_folder('../../nbs/knowledge_base/')


In [300]:
def create_folder(folder_path):
    if os.path.exists(folder_path):
        return

    os.mkdir(folder_path)


def create_category(crumb, base_output_folder, listing_template , posts_folder="posts"):


    path = crumb['path']
    path.replace('knowledge_base', '')
    
    listing_title = 'Knowledge Base' if path == '' else crumb.get('title')
    listing_description = crumb.get("description")

    output_folder = os.path.join(base_output_folder, path)

    create_folder(output_folder)

    create_folder(os.path.join(output_folder, posts_folder))

    if listing_title:       
        listing_template = listing_template.replace(
            "<<TITLE>>", listing_title).replace('<<FOLDER>>', f'"{posts_folder}"')
        listing_template = listing_template.replace(
            "<<SUBTITLE>>", listing_description) if listing_description else listing_template.replace('subtitle: <<SUBTITLE>>', '')

        listing_path = os.path.join(output_folder, 'index.qmd')

        with open(listing_path, 'w', encoding = "utf-8") as f:
            f.write(listing_template)

with open('template_listing.txt', 'r', encoding="utf-8") as f:
    listing_template = f.read()

res = [create_category(crumb, base_output_folder=OUTPUT_FOLDER, listing_template = listing_template) for crumb in crumb_ls]

# add_to_yaml

In [19]:
import yaml


def update_yaml():
    yaml_path = '../../nbs/_quarto.yml'

    with open(yaml_path, 'r') as f:
        quarto_yaml = yaml.safe_load(f)

    from pprint import pprint
    pprint(quarto_yaml)

    quarto_yaml['website']['sidebar'].update({'contents': [
        {'auto': "index.ipynb"},
        * add_to_yaml
    ]})

    with open(yaml_path, 'w') as f:
        yaml.dump(quarto_yaml, f)
        
    return quarto_yaml


# update_yaml()


{'format': {'html': {'css': 'styles.css', 'theme': 'cosmo', 'toc': True}},
 'metadata-files': ['nbdev.yml'],
 'project': {'type': 'website'},
 'website': {'navbar': {'background': 'primary', 'search': True},
             'open-graph': True,
             'repo-actions': ['issue'],
             'sidebar': {'contents': [{'auto': 'index.ipynb'},
                                      {'contents': ['knowledge_base/administrating_domo/index.qmd'],
                                       'section': 'Administrating Domo'},
                                      {'contents': ['knowledge_base/connecting_data_to_domo/index.qmd'],
                                       'section': 'Connecting Data To Domo'},
                                      {'contents': ['knowledge_base/consuming_data_in_domo/index.qmd'],
                                       'section': 'Consuming Data In Domo'},
                                      {'contents': ['knowledge_base/creating_content_in_domo/index.qmd'],
           

{'format': {'html': {'css': 'styles.css', 'theme': 'cosmo', 'toc': True}},
 'metadata-files': ['nbdev.yml'],
 'project': {'type': 'website'},
 'website': {'navbar': {'background': 'primary', 'search': True},
  'open-graph': True,
  'repo-actions': ['issue'],
  'sidebar': {'contents': [{'auto': 'index.ipynb'},
    {'section': 'Administrating Domo',
     'contents': ['knowledge_base/administrating_domo/index.qmd']},
    {'section': 'Connecting Data To Domo',
     'contents': ['knowledge_base/connecting_data_to_domo/index.qmd']},
    {'section': 'Consuming Data In Domo',
     'contents': ['knowledge_base/consuming_data_in_domo/index.qmd']},
    {'section': 'Creating Content In Domo',
     'contents': ['knowledge_base/creating_content_in_domo/index.qmd']},
    {'section': 'Creating Custom Visualizations',
     'contents': ['knowledge_base/creating_custom_visualizations/index.qmd']},
    {'section': 'Distributing Domo Content',
     'contents': ['knowledge_base/distributing_domo_content/ind